In [39]:
# Imports
import numpy as np
import pandas as pd
import respy as rp
from estimagic.optimization.optimize import minimize

In [3]:
# Parameters for the simulation --> will go to config.py
NUM_AGENTS = 500
NUM_PERIODS = 5

In the following we will simulate 500 representative Robinson Crusoe agents that live and plan for 5 periods.

In [4]:
params_base, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)
options["n_periods"] = NUM_PERIODS
options["simulation_agents"] = NUM_AGENTS

**Simulation**
1. Build the simulate function
2. Enter the parameters into the built simulate function and simulate the agents

In [6]:
simulate = rp.get_simulate_func(params_base, options)
df_base = simulate(params_base)

The observed data will show that our 500th agent will alternate between hammock and fishing. In the first two periods he will idle around in the hammock. In period 2 he starts to go fishing in order to make a living. However, in the last period our Robinson No. 500 decides to go into retirement in the hammock.

In [8]:
df_base.tail(10)

Experience_Fishing Lagged_Choice_1  Shock_Reward_Fishing  \
Identifier Period                                                             
498        0                        0         hammock              0.348941   
           1                        0         hammock              1.829732   
           2                        1         fishing              0.415493   
           3                        1         hammock              0.469027   
           4                        1         hammock              0.353453   
499        0                        0         hammock              0.169878   
           1                        0         hammock              5.522572   
           2                        1         fishing              4.854371   
           3                        2         fishing              0.493262   
           4                        2         hammock              0.419093   

                   Meas_Error_Wage_Fishing  Shock_Reward_Hammock  \
Identifier Period                                                  
498        0                             1              1.468248   
           1                             1              0.159167   
           2                             1              1.397043   
           3                             1             -0.250270   
           4                             1             -0.182402   
499        0                             1              1.889332   
           1                             1             -0.504682   
           2                             1              0.023921   
           3                             1              0.544756   
           4                             1              0.376626   

                   Meas_Error_Wage_Hammock   Choice      Wage  Discount_Rate  \
Identifier Period                                                              
498        0                             1  hammock       NaN           0.95   
           1                             1  fishing  1.829732           0.95   
           2                             1  hammock       NaN           0.95   
           3                             1  hammock       NaN           0.95   
           4                             1  hammock       NaN           0.95   
499        0                             1  hammock       NaN           0.95   
           1                             1  fishing  5.522572           0.95   
           2                             1  fishing  5.364910           0.95   
           3                             1  hammock       NaN           0.95   
           4                             1  hammock       NaN           0.95   

                   Nonpecuniary_Reward_Fishing  Wage_Fishing  \
Identifier Period                                              
498        0                                -1      0.348941   
           1                                -1      1.829732   
           2                                -1      0.459191   
           3                                -1      0.518355   
           4                                -1      0.390626   
499        0                                -1      0.169878   
           1                                -1      5.522572   
           2                                -1      5.364910   
           3                                -1      0.602471   
           4                                -1      0.511881   

                   Flow_Utility_Fishing  Value_Function_Fishing  \
Identifier Period                                                 
498        0                  -0.651059                8.350542   
           1                   0.829732                7.810173   
           2                  -0.540809                4.526047   
           3                  -0.481645                2.388202   
           4                  -0.609374               -0.609374   
499        0                  -0.830122               

In [9]:
def calc_choice_frequencies(df):
    """Calculation of choice frequencies"""
    return df.groupby("Period").Choice.value_counts(normalize=True).unstack()
    

In [10]:
def calc_wage_distribution(df):
    """Calculation of wage distribution"""
    return df.groupby(["Period"])["Wage"].describe()[["mean", "std"]]

In [11]:
calc_moments = {"Choice Frequencies":calc_choice_frequencies, 
                "Wage Distribution":calc_wage_distribution}

In [12]:
def replace_nans(df):
    return df.fillna(0)

In [15]:
moments_obs = {"Choice Frequencies": replace_nans(calc_moments["Choice Frequencies"](df_base)),
               "Wage Distribution": replace_nans(calc_moments["Wage Distribution"](df_base))}

In [16]:
print('Choice Frequencies')
print(moments_obs["Choice Frequencies"])
print('\n Wage Distribution')
print(moments_obs["Wage Distribution"])

Choice Frequencies
Choice  fishing  hammock
Period                  
0         0.406    0.594
1         0.326    0.674
2         0.382    0.618
3         0.390    0.610
4         0.242    0.758

 Wage Distribution
            mean       std
Period                    
0       2.734174  2.374415
1       3.515533  3.474489
2       3.351965  3.075048
3       3.813647  3.424537
4       5.294609  5.459624


In [17]:
def get_weighting_matrix(data, calc_moments, num_boots, num_agents_msm):
    """ Compute weighting matrix for estimation with MSM."""
    # Seed for reproducibility.
    np.random.seed(123)

    index_base = data.index.get_level_values("Identifier").unique()
    
    # Create bootstrapped moments.
    moments_sample = list()
    for _ in range(num_boots):
        ids_boot = np.random.choice(index_base, num_agents_msm, replace=False)
        moments_boot = [calc_moments[key](data.loc[ids_boot, :]) for key in calc_moments.keys()]

        moments_boot = rp.get_flat_moments(moments_boot)

        moments_sample.append(moments_boot)
    
    # Compute variance for each moment and construct diagonal weighting matrix.
    moments_var = np.array(moments_sample).var(axis=0)
    weighting_matrix = np.diag(moments_var ** (-1))

    return np.nan_to_num(weighting_matrix)

In [21]:
criterion_msm_base = rp.get_msm_func(params_base, options, calc_moments, replace_nans, moments_obs, W)

In [23]:
fval = criterion_msm_base(params_base)
fval

0.0

Get the MSM criterion function from respy.

In [32]:
params_delta = params_base.copy()
params_delta.loc["delta", "value"] = 0.8

In [33]:
simulate_delta = rp.get_simulate_func(params_delta, options)
df_sim_delta = simulate_delta(params_delta)

In [34]:
moments_sim_delta = {"Choice Frequencies": replace_nans(calc_moments["Choice Frequencies"](df_sim_delta)),
               "Wage Distribution": replace_nans(calc_moments["Wage Distribution"](df_sim_delta))}

In [35]:
criterion_msm_delta = rp.get_msm_func(params_delta, options, calc_moments, replace_nans, moments_obs, W)

In [36]:
fval = criterion_msm_delta(params_delta)
fval

2.670162819888279e+28

In [37]:
params_eta = params_base.copy()
params_eta.loc[("eta", "eta"), :] = 0.1 #eta_values["baseline"]
#params_eta

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00
eta                     eta                       0.10

ESTIMATION

In [40]:
criterion_msm_eta = rp.get_msm_func(params_eta, options, calc_moments, replace_nans, moments_obs, W)

In [42]:
rslt = minimize(
    criterion = criterion_msm_eta,
    params = params_eta, 
    algorithm = "nlopt_bobyqa"
)

In [44]:
#rslt

ESTIMATION FIXING CONSTRAINTS

In [28]:
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"}, 
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type":"fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

In [29]:
constr_eta = constr_base.copy()
constr

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00